学习 [简单粗暴 TensorFlow 2](https://tf.wiki/zh_hans/) (github页 https://github.com/snowkylin/tensorflow-handbook)

In [1]:
import numpy as np
import tensorflow as tf
import datetime
import os 
import sys
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 80
pd.options.display.precision = 4
pd.options.display.max_rows = 999
pd.options.display.float_format = '{:.4f}'.format  # 防止科学计数法，小数显示4位

In [2]:
# 定义一个随机数（标量）
random_float = tf.random.uniform(shape=())

# 定义一个有2个元素的零向量
zero_vector = tf.zeros(shape=(2))

In [3]:
random_float

<tf.Tensor: shape=(), dtype=float32, numpy=0.6756563>

In [4]:
zero_vector

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0., 0.], dtype=float32)>

In [5]:
# 定义两个2×2的常量矩阵
A = tf.constant([[1., 2.], [3., 4.]])
B = tf.constant([[5., 6.], [7., 8.]])

In [6]:
print(A.shape)
print(A.dtype)
print(A.numpy())

(2, 2)
<dtype: 'float32'>
[[1. 2.]
 [3. 4.]]


In [7]:
C = tf.add(A, B)
D = tf.matmul(A, B)

In [8]:
C
D

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[ 6.,  8.],
       [10., 12.]], dtype=float32)>

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[19., 22.],
       [43., 50.]], dtype=float32)>

In [9]:
x = tf.Variable(initial_value=3.)
with tf.GradientTape() as tape:     # 在 tf.GradientTape() 的上下文内，所有计算步骤都会被记录以用于求导
    y = tf.square(x)
y_grad = tape.gradient(y, x)        # 计算y关于x的导数
print(y, y_grad, sep='\n')

tf.Tensor(9.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)


In [10]:
X = tf.constant([[1., 2.], [3., 4.]])
y = tf.constant([[1.], [2.]])
w = tf.Variable(initial_value=[[1.], [2.]])
b = tf.Variable(initial_value=1.)
with tf.GradientTape() as tape:
    L = tf.reduce_sum(tf.square(tf.matmul(X, w) + b - y))
w_grad, b_grad = tape.gradient(L, [w, b])        # 计算L(w, b)关于w, b的偏导数
print(L, w_grad, b_grad, sep = '\n')

tf.Tensor(125.0, shape=(), dtype=float32)
tf.Tensor(
[[ 70.]
 [100.]], shape=(2, 1), dtype=float32)
tf.Tensor(30.0, shape=(), dtype=float32)


In [11]:

X_raw = np.array([2013, 2014, 2015, 2016, 2017], dtype=np.float32)
y_raw = np.array([12000, 14000, 15000, 16500, 17500], dtype=np.float32)

X = (X_raw - X_raw.min()) / (X_raw.max() - X_raw.min())
y = (y_raw - y_raw.min()) / (y_raw.max() - y_raw.min())

In [12]:
X
y

array([0.  , 0.25, 0.5 , 0.75, 1.  ], dtype=float32)

array([0.        , 0.36363637, 0.54545456, 0.8181818 , 1.        ],
      dtype=float32)

### 使用Numpy线性回归

In [13]:
a, b = 0, 0

num_epoch = 10000
learning_rate = 5e-4
for e in range(num_epoch):
    # 手动计算损失函数关于自变量（模型参数）的梯度
    y_pred = a * X + b
    grad_a, grad_b = 2 * (y_pred - y).dot(X), 2 * (y_pred - y).sum()

    # 更新参数
    a, b = a - learning_rate * grad_a, b - learning_rate * grad_b

print(a, b)

0.9763702027872221 0.057564988311377796


### 尝试Tensorflow线性回归

In [22]:
X = tf.constant(X)
y = tf.constant(y)

a = tf.Variable(initial_value=0.)
b = tf.Variable(initial_value=0.)
variables = [a, b]

num_epoch = 10000
optimizer = tf.keras.optimizers.SGD(learning_rate=5e-4)
for e in range(num_epoch):
    # 使用tf.GradientTape()记录损失函数的梯度信息
    with tf.GradientTape() as tape:
        y_pred = a * X + b
        loss = tf.reduce_sum(tf.square(y_pred - y))
    # TensorFlow自动计算损失函数关于自变量（模型参数）的梯度
    grads = tape.gradient(loss, variables)
    # TensorFlow自动根据梯度更新参数
    _ = optimizer.apply_gradients(grads_and_vars=zip(grads, variables))

In [23]:
variables

[<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.97637>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.057565063>]

In [16]:
c = 1
d = 1 
alist = [c, d]

In [17]:
alist

[1, 1]

In [18]:
d = 2
alist

[1, 1]

### Model与Layer

In [25]:
import tensorflow as tf

X = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
y = tf.constant([[10.0], [20.0]])


class Linear(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense = tf.keras.layers.Dense(
            units=1,
            activation=None,
            kernel_initializer=tf.zeros_initializer(),
            bias_initializer=tf.zeros_initializer()
        )

    def call(self, input):
        output = self.dense(input)
        return output


# 以下代码结构与前节类似
model = Linear()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
for i in range(10000):
    with tf.GradientTape() as tape:
        y_pred = model(X)      # 调用模型 y_pred = model(X) 而不是显式写出 y_pred = a * X + b
        loss = tf.reduce_mean(tf.square(y_pred - y))
    grads = tape.gradient(loss, model.variables)    # 使用 model.variables 这一属性直接获得模型中的所有变量
    _ = optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
print(model.variables)

[<tf.Variable 'linear_1/dense_1/kernel:0' shape=(3, 1) dtype=float32, numpy=
array([[4.8735565e-06],
       [1.1111153e+00],
       [2.2222164e+00]], dtype=float32)>, <tf.Variable 'linear_1/dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([1.1111081], dtype=float32)>]


<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=10000>

### 基础示例：多层感知机(MLP)

In [28]:
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的图像默认为uint8（0-255的数字）。以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 从数据集中随机取出batch_size个元素并返回
        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]

In [29]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()    # Flatten层将除第一维（batch_size）以外的维度展平
        self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    def call(self, inputs):         # [batch_size, 28, 28, 1]
        x = self.flatten(inputs)    # [batch_size, 784]
        x = self.dense1(x)          # [batch_size, 100]
        x = self.dense2(x)          # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output

In [30]:
num_epochs = 5
batch_size = 50
learning_rate = 0.001

In [31]:
model = MLP()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [33]:
num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        print("batch %d: loss %f" % (batch_index, loss.numpy()))
    grads = tape.gradient(loss, model.variables)
    _ = optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

batch 0: loss 0.027174
batch 1: loss 0.053722
batch 2: loss 0.037445
batch 3: loss 0.040360
batch 4: loss 0.113464
batch 5: loss 0.101838
batch 6: loss 0.056415
batch 7: loss 0.039312
batch 8: loss 0.053956
batch 9: loss 0.021681
batch 10: loss 0.085016
batch 11: loss 0.004939
batch 12: loss 0.019586
batch 13: loss 0.017728
batch 14: loss 0.035495
batch 15: loss 0.027303
batch 16: loss 0.026122
batch 17: loss 0.056961
batch 18: loss 0.092513
batch 19: loss 0.028139
batch 20: loss 0.020823
batch 21: loss 0.044066
batch 22: loss 0.145612
batch 23: loss 0.032172
batch 24: loss 0.071870
batch 25: loss 0.190534
batch 26: loss 0.093715
batch 27: loss 0.008391
batch 28: loss 0.048420
batch 29: loss 0.019753
batch 30: loss 0.006633
batch 31: loss 0.039991
batch 32: loss 0.063499
batch 33: loss 0.105380
batch 34: loss 0.155985
batch 35: loss 0.009560
batch 36: loss 0.027525
batch 37: loss 0.059563
batch 38: loss 0.011615
batch 39: loss 0.051134
batch 40: loss 0.055502
batch 41: loss 0.079165
ba

batch 346: loss 0.154429
batch 347: loss 0.017737
batch 348: loss 0.073637
batch 349: loss 0.215291
batch 350: loss 0.010954
batch 351: loss 0.051547
batch 352: loss 0.014291
batch 353: loss 0.011932
batch 354: loss 0.075875
batch 355: loss 0.064897
batch 356: loss 0.009475
batch 357: loss 0.091193
batch 358: loss 0.013639
batch 359: loss 0.016796
batch 360: loss 0.027518
batch 361: loss 0.023205
batch 362: loss 0.164686
batch 363: loss 0.011505
batch 364: loss 0.036221
batch 365: loss 0.004177
batch 366: loss 0.020029
batch 367: loss 0.033472
batch 368: loss 0.023946
batch 369: loss 0.005163
batch 370: loss 0.068575
batch 371: loss 0.049143
batch 372: loss 0.090909
batch 373: loss 0.027526
batch 374: loss 0.042742
batch 375: loss 0.108374
batch 376: loss 0.008816
batch 377: loss 0.036353
batch 378: loss 0.143558
batch 379: loss 0.045974
batch 380: loss 0.020335
batch 381: loss 0.072126
batch 382: loss 0.037960
batch 383: loss 0.165111
batch 384: loss 0.023666
batch 385: loss 0.057682


batch 682: loss 0.004848
batch 683: loss 0.019753
batch 684: loss 0.125232
batch 685: loss 0.042461
batch 686: loss 0.046523
batch 687: loss 0.034602
batch 688: loss 0.182959
batch 689: loss 0.063681
batch 690: loss 0.084642
batch 691: loss 0.010894
batch 692: loss 0.059165
batch 693: loss 0.015719
batch 694: loss 0.039780
batch 695: loss 0.039223
batch 696: loss 0.060472
batch 697: loss 0.049201
batch 698: loss 0.036549
batch 699: loss 0.012944
batch 700: loss 0.011723
batch 701: loss 0.027787
batch 702: loss 0.002452
batch 703: loss 0.025258
batch 704: loss 0.122908
batch 705: loss 0.013927
batch 706: loss 0.118726
batch 707: loss 0.032002
batch 708: loss 0.012810
batch 709: loss 0.183252
batch 710: loss 0.041288
batch 711: loss 0.044542
batch 712: loss 0.043158
batch 713: loss 0.009210
batch 714: loss 0.031241
batch 715: loss 0.095207
batch 716: loss 0.040645
batch 717: loss 0.004063
batch 718: loss 0.017087
batch 719: loss 0.027027
batch 720: loss 0.022512
batch 721: loss 0.022109


batch 1018: loss 0.049235
batch 1019: loss 0.054541
batch 1020: loss 0.019808
batch 1021: loss 0.018229
batch 1022: loss 0.017634
batch 1023: loss 0.075808
batch 1024: loss 0.028664
batch 1025: loss 0.018941
batch 1026: loss 0.022299
batch 1027: loss 0.029203
batch 1028: loss 0.043654
batch 1029: loss 0.151648
batch 1030: loss 0.031644
batch 1031: loss 0.026843
batch 1032: loss 0.027090
batch 1033: loss 0.007005
batch 1034: loss 0.062424
batch 1035: loss 0.082140
batch 1036: loss 0.008871
batch 1037: loss 0.064578
batch 1038: loss 0.027467
batch 1039: loss 0.212468
batch 1040: loss 0.029698
batch 1041: loss 0.022028
batch 1042: loss 0.026772
batch 1043: loss 0.039174
batch 1044: loss 0.006890
batch 1045: loss 0.041393
batch 1046: loss 0.034018
batch 1047: loss 0.006918
batch 1048: loss 0.004376
batch 1049: loss 0.009802
batch 1050: loss 0.041921
batch 1051: loss 0.095212
batch 1052: loss 0.011746
batch 1053: loss 0.072940
batch 1054: loss 0.057788
batch 1055: loss 0.009074
batch 1056: 

batch 1345: loss 0.013628
batch 1346: loss 0.037386
batch 1347: loss 0.020322
batch 1348: loss 0.080919
batch 1349: loss 0.014107
batch 1350: loss 0.011195
batch 1351: loss 0.068403
batch 1352: loss 0.011223
batch 1353: loss 0.151989
batch 1354: loss 0.044409
batch 1355: loss 0.026952
batch 1356: loss 0.033373
batch 1357: loss 0.015391
batch 1358: loss 0.108527
batch 1359: loss 0.037941
batch 1360: loss 0.006855
batch 1361: loss 0.037990
batch 1362: loss 0.088026
batch 1363: loss 0.078235
batch 1364: loss 0.003535
batch 1365: loss 0.046135
batch 1366: loss 0.073160
batch 1367: loss 0.046077
batch 1368: loss 0.023011
batch 1369: loss 0.038801
batch 1370: loss 0.020895
batch 1371: loss 0.017901
batch 1372: loss 0.037317
batch 1373: loss 0.020662
batch 1374: loss 0.075727
batch 1375: loss 0.219017
batch 1376: loss 0.225964
batch 1377: loss 0.081201
batch 1378: loss 0.022470
batch 1379: loss 0.018435
batch 1380: loss 0.018522
batch 1381: loss 0.033465
batch 1382: loss 0.009728
batch 1383: 

...

batch 4570: loss 0.005335
batch 4571: loss 0.000651
batch 4572: loss 0.022285
batch 4573: loss 0.072639
batch 4574: loss 0.035801
batch 4575: loss 0.029677
batch 4576: loss 0.017971
batch 4577: loss 0.006688
batch 4578: loss 0.019029
batch 4579: loss 0.024749
batch 4580: loss 0.010227
batch 4581: loss 0.019319
batch 4582: loss 0.079775
batch 4583: loss 0.100155
batch 4584: loss 0.092702
batch 4585: loss 0.070334
batch 4586: loss 0.015007
batch 4587: loss 0.013493
batch 4588: loss 0.019800
batch 4589: loss 0.039612
batch 4590: loss 0.008423
batch 4591: loss 0.011145
batch 4592: loss 0.009940
batch 4593: loss 0.011571
batch 4594: loss 0.068499
batch 4595: loss 0.052960
batch 4596: loss 0.010731
batch 4597: loss 0.163809
batch 4598: loss 0.010237
batch 4599: loss 0.064604
batch 4600: loss 0.006499
batch 4601: loss 0.006417
batch 4602: loss 0.011987
batch 4603: loss 0.065023
batch 4604: loss 0.012093
batch 4605: loss 0.056215
batch 4606: loss 0.025775
batch 4607: loss 0.008937
batch 4608: 

batch 4917: loss 0.056275
batch 4918: loss 0.140094
batch 4919: loss 0.014832
batch 4920: loss 0.099335
batch 4921: loss 0.004263
batch 4922: loss 0.041694
batch 4923: loss 0.028060
batch 4924: loss 0.175260
batch 4925: loss 0.027695
batch 4926: loss 0.056697
batch 4927: loss 0.042701
batch 4928: loss 0.011173
batch 4929: loss 0.008568
batch 4930: loss 0.003489
batch 4931: loss 0.042401
batch 4932: loss 0.005748
batch 4933: loss 0.021833
batch 4934: loss 0.001731
batch 4935: loss 0.004522
batch 4936: loss 0.032046
batch 4937: loss 0.009649
batch 4938: loss 0.008478
batch 4939: loss 0.139362
batch 4940: loss 0.046747
batch 4941: loss 0.019312
batch 4942: loss 0.002374
batch 4943: loss 0.007552
batch 4944: loss 0.006226
batch 4945: loss 0.053583
batch 4946: loss 0.006304
batch 4947: loss 0.014438
batch 4948: loss 0.072245
batch 4949: loss 0.011468
batch 4950: loss 0.001975
batch 4951: loss 0.072208
batch 4952: loss 0.050529
batch 4953: loss 0.021710
batch 4954: loss 0.042880
batch 4955: 

batch 5266: loss 0.010639
batch 5267: loss 0.005159
batch 5268: loss 0.011611
batch 5269: loss 0.034774
batch 5270: loss 0.008198
batch 5271: loss 0.004184
batch 5272: loss 0.009277
batch 5273: loss 0.012570
batch 5274: loss 0.036237
batch 5275: loss 0.040733
batch 5276: loss 0.026381
batch 5277: loss 0.032353
batch 5278: loss 0.063596
batch 5279: loss 0.007703
batch 5280: loss 0.052270
batch 5281: loss 0.082478
batch 5282: loss 0.019913
batch 5283: loss 0.015543
batch 5284: loss 0.052177
batch 5285: loss 0.005066
batch 5286: loss 0.004138
batch 5287: loss 0.034828
batch 5288: loss 0.072946
batch 5289: loss 0.007886
batch 5290: loss 0.013298
batch 5291: loss 0.023339
batch 5292: loss 0.026843
batch 5293: loss 0.023509
batch 5294: loss 0.069019
batch 5295: loss 0.017305
batch 5296: loss 0.042788
batch 5297: loss 0.055065
batch 5298: loss 0.007446
batch 5299: loss 0.020590
batch 5300: loss 0.004678
batch 5301: loss 0.009577
batch 5302: loss 0.032395
batch 5303: loss 0.021803
batch 5304: 

batch 5591: loss 0.004674
batch 5592: loss 0.003827
batch 5593: loss 0.021927
batch 5594: loss 0.002634
batch 5595: loss 0.007584
batch 5596: loss 0.012809
batch 5597: loss 0.004530
batch 5598: loss 0.002025
batch 5599: loss 0.004796
batch 5600: loss 0.062279
batch 5601: loss 0.082775
batch 5602: loss 0.025398
batch 5603: loss 0.025844
batch 5604: loss 0.004766
batch 5605: loss 0.019091
batch 5606: loss 0.048089
batch 5607: loss 0.015933
batch 5608: loss 0.004664
batch 5609: loss 0.014718
batch 5610: loss 0.017375
batch 5611: loss 0.003807
batch 5612: loss 0.005718
batch 5613: loss 0.011243
batch 5614: loss 0.013858
batch 5615: loss 0.027359
batch 5616: loss 0.102820
batch 5617: loss 0.044013
batch 5618: loss 0.001655
batch 5619: loss 0.014798
batch 5620: loss 0.002931
batch 5621: loss 0.017120
batch 5622: loss 0.065747
batch 5623: loss 0.039446
batch 5624: loss 0.003116
batch 5625: loss 0.006188
batch 5626: loss 0.014805
batch 5627: loss 0.027907
batch 5628: loss 0.094886
batch 5629: 

batch 5932: loss 0.019662
batch 5933: loss 0.046207
batch 5934: loss 0.011227
batch 5935: loss 0.004120
batch 5936: loss 0.008962
batch 5937: loss 0.061059
batch 5938: loss 0.001567
batch 5939: loss 0.004447
batch 5940: loss 0.007740
batch 5941: loss 0.019300
batch 5942: loss 0.003518
batch 5943: loss 0.011017
batch 5944: loss 0.032036
batch 5945: loss 0.012708
batch 5946: loss 0.007838
batch 5947: loss 0.012349
batch 5948: loss 0.001157
batch 5949: loss 0.017833
batch 5950: loss 0.001841
batch 5951: loss 0.025660
batch 5952: loss 0.025307
batch 5953: loss 0.020590
batch 5954: loss 0.005130
batch 5955: loss 0.010181
batch 5956: loss 0.002960
batch 5957: loss 0.009564
batch 5958: loss 0.019505
batch 5959: loss 0.003893
batch 5960: loss 0.029474
batch 5961: loss 0.011348
batch 5962: loss 0.001135
batch 5963: loss 0.021151
batch 5964: loss 0.010434
batch 5965: loss 0.009614
batch 5966: loss 0.006904
batch 5967: loss 0.015484
batch 5968: loss 0.022732
batch 5969: loss 0.046728
batch 5970: 